In [ ]:
from __future__ import print_function

import pennylane as qml
import tensorflow as tf
print(tf.__version__)
import numpy as np
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os

2.14.0


In [ ]:
batch_size = 32
num_classes = 10
epochs = 50
data_augmentation = True
num_predictions = 20

save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

## Loading data...

In [ ]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Filter only classes 3 and 88
class_indices_train = np.where((y_train[:, 0] == 3) | (y_train[:, 0] == 88))[0]
class_indices_test = np.where((y_test[:, 0] == 3) | (y_test[:, 0] == 88))[0]

x_train = x_train[class_indices_train]
y_train = y_train[class_indices_train]

x_test = x_test[class_indices_test]
y_test = y_test[class_indices_test]

# Map class labels to consecutive integers starting from 0
class_mapping = {3: 0, 88: 1}
y_train_mapped = np.vectorize(class_mapping.get)(y_train[:, 0])
y_test_mapped = np.vectorize(class_mapping.get)(y_test[:, 0])

# Print information about the new dataset
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
num_classes = 2  # Number of classes (3 and 88)
y_train = tf.keras.utils.to_categorical(y_train_mapped, num_classes)
y_test = tf.keras.utils.to_categorical(y_test_mapped, num_classes)


x_train shape: (1000, 32, 32, 3)
1000 train samples
200 test samples


## Forward Propagation
+ **Building a training model with `tf.keras`**

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:])) # feature map:(32x32x32)
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))  # feature map:(30x30x32)
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))  # feature map:(30x30x32)
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))  # feature map:(15x15x32)
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))  # feature map:(15x15x64)
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same'))  # feature map:(15x15x64)
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))  # feature map:(13x13x64)
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))  # feature map:(6x6x64)
model.add(Dropout(0.25))

model.add(Flatten())  # (2304,)
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights_0, weight_1):
    qml.RX(inputs[0], wires=0)
    qml.RX(inputs[1], wires=1)
    qml.Rot(*weights_0, wires=0)
    qml.RY(weight_1, wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))

weight_shapes = {"weights_0": 3, "weight_1": 1}
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=2)

clayer = tf.keras.layers.Dense(2)

## Backpropagation

In [ ]:
# initiate RMSprop optimizer
opt = tf.keras.optimizers.legacy.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

## Normalization

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Data Augmentation

In [ ]:
import os

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


Physical devices cannot be modified after being initialized


In [ ]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit(x_train, y_train, batch_size=4, epochs=epochs, validation_data=(x_test, y_test), workers=4)


Using real-time data augmentation.
Epoch 1/50
250/250 [==============================] - 6s 16ms/step - loss: 0.6929 - accuracy: 0.5080 - val_loss: 0.6881 - val_accuracy: 0.5200
Epoch 2/50
250/250 [==============================] - 2s 8ms/step - loss: 0.6821 - accuracy: 0.6000 - val_loss: 0.6764 - val_accuracy: 0.7100
Epoch 3/50
250/250 [==============================] - 1s 6ms/step - loss: 0.6576 - accuracy: 0.7050 - val_loss: 0.6481 - val_accuracy: 0.6850
Epoch 4/50
250/250 [==============================] - 1s 6ms/step - loss: 0.6125 - accuracy: 0.7350 - val_loss: 0.6059 - val_accuracy: 0.7100
Epoch 5/50
250/250 [==============================] - 1s 6ms/step - loss: 0.5632 - accuracy: 0.7540 - val_loss: 0.5672 - val_accuracy: 0.7150
Epoch 6/50
250/250 [==============================] - 1s 6ms/step - loss: 0.5306 - accuracy: 0.7520 - val_loss: 0.5412 - val_accuracy: 0.7150
Epoch 7/50
250/250 [==============================] - 1s 6ms/step - loss: 0.4991 - accuracy: 0.7680 - val_loss: 

## Saving the model...

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /content/saved_models/keras_cifar10_trained_model.h5 


## Evaluation

In [ ]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

7/7 - 0s - loss: 0.2248 - accuracy: 0.9000 - 184ms/epoch - 26ms/step
Test loss: 0.22477000951766968
Test accuracy: 0.8999999761581421
